# EP4 - Simulação paralelizada de sólidos usando DEM

Nota: todos os nomes de funções e etc se encontram em inglês. Isso foi feito por conta do próprio python, que é inteiramente em inglês. 

## Preâmbulo

In [3]:
import math
import numpy as np
import pandas as pd
from numpy.linalg import norm
from itertools import combinations

In [4]:
# Algumas funções geométricas auxiliares

def vec(x,y,z):
    '''Função para facilmente transformar coordenadas em um vetor numpy'''
    return np.array([x,y,z])

def plane_point_distance(p, p1, n):
    '''Função que calcula a distância de um ponto p até um plano
    definido por um ponto p1 e um vetor normal n'''
    dist = np.abs(np.dot(p-p1, n))
    return dist

def rotation_3d(vec, rot):
    '''Função que rotaciona um vetor 3d em torno dos eixos x,y,z, a partir
    de um vetor de rotação rot = [rot_x, rot_y, rot_z]. Nota: @ é um operador
    da biblioteca numpy que realiza a multiplicação de matrizes.'''
    if rot[0] != 0:
        Rx = np.array([[1,0,0],
                       [0,math.cos(rot[0]),-math.sin(rot[0])],
                       [0,math.sin(rot[0]),math.cos(rot[0])]])
        vec = Rx @ vec
    
    if rot[1] != 0:
        Ry = np.array([[math.cos(rot[1]),0,math.sin(rot[1])],
                       [0,1,0],
                       [-math.sin(rot[1]),0,math.cos(rot[1])]])
        vec = Ry @ vec

    if rot[2] != 0:
        Rz = np.array([[math.cos(rot[2]),-math.sin(rot[2]),0],
                       [math.sin(rot[2]),math.cos(rot[2]),0],
                       [0,0,1]])
        vec = Rz @ vec 
        
    return vec

In [5]:
# Classe das Partículas (Grãos)

'''Classe que define uma partícula, que aqui chamo de grain (grão) por simplicidade.
define um grão por sua posição, raio, massa, velocidade, aceleração e força. Para 
gerenciar os grãos, é sempre criada uma lista de grãos, grain_list, que é argumento
de todas as funções que gerenciam as interações que envolvem os grãos.'''

class grain: 
    def __init__(self, pos, radius, density, vel): 
        self.radius = float(radius)
        x,y,z = pos
        self.pos     = vec(x,y,z)
        self.mass    = density*math.pi*(self.radius**3)*(4/3)
        self.vel     = vel
        self.acc     = vec(0.,0.,0.)
        self.force   = vec(0.,0.,0.)

In [6]:
test = np.array([[1,2,3],[4,5,6],[7,8,9]])

In [7]:
test

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [8]:
test.T

array([[1, 4, 7],
       [2, 5, 8],
       [3, 6, 9]])

In [9]:
# Funções de gerenciamento de forças e interações

def cube_walls(center, scale, grain_list, K, rot = vec(0,0,0)):
    '''Função que gerencia a geometria do cubo, e a força de contato nas 
    partículas que estão em contato com as paredes do cubo.'''

    x,y,z = center

    vectors = [vec(1,0,0), vec(0,1,0), vec(0,0,1)]

    if not rot.all == 0:
        rot_vectors = [rotation_3d(x, rot) for x in vectors]
        vectors = rot_vectors.copy()
    
    n_vectors = [x/norm(x) for x in vectors]
    vectors = n_vectors.copy()

    t_matrix = np.array(vectors).T

    inv_matrix = np.linalg.inv(t_matrix)

    for grain in grain_list:

        p = grain.pos

        p = t_matrix @ p

        if p[0] > x + scale - grain.radius:
            p[0] = x +  scale - grain.radius
            grain.vel[0] = -grain.vel[0]*K

        if p[0] < x - scale + grain.radius:
            p[0] = x - scale + grain.radius
            grain.vel[0] = -grain.vel[0]*K
        
        if p[1] > y + scale - grain.radius:
            p[1] = y + scale - grain.radius
            grain.vel[1] = -grain.vel[1]*K
        
        if p[1] < y - scale + grain.radius:
            p[1] = y - scale + grain.radius
            grain.vel[1] = -grain.vel[1]*K

        if p[2] > z + scale - grain.radius:
            p[2] = z + scale - grain.radius
            grain.vel[2] = -grain.vel[2]*K
        
        if p[2] < z - scale + grain.radius:
            p[2] = z - scale + grain.radius
            grain.vel[2] = -grain.vel[2]*K

        p = inv_matrix @ p

        grain.pos = p




def contact(gr1, gr2):
    '''Função que gerencia as forças de contato entre dois grãos.'''

    ks = 1e3
    kd = 1e3
    kt = 0.5
    mu = 0.2

    # Força Normal
    
    csi = max(0, gr1.radius + gr2.radius - norm(gr1.pos - gr2.pos))

    N = gr1.pos - gr2.pos

    Fs = ks*csi*N/norm(N)

    Fd = kd*N

    Fn = Fs + Fd

    gr1.force += Fn
    gr2.force -= Fn

    # Força Tangencial

    Vt = gr1.vel - gr2.vel

    if Vt.all != 0:
        Ft = -min(mu*norm(Fn), kt*norm(Vt))*Vt/norm(Vt)

        gr1.force += Ft
        gr2.force -= Ft


def apply_gravity(grain_list, g):
    '''Função que gerencia a força gravitacional aplicada somente aos grãos.'''
    for gr in grain_list:
        gr.force += gr.mass*g

def reset_force(grain_list):
    '''Função que coloca as forças de todos os grãos em zero. A finalidade
    dessa função é simples: se as forças não forem todas colocadas em zero
    no início de cada iteração, forças como a gravidade iriam se acumular
    de uma iteração para a outra, algo não desejado.'''
    for gr in grain_list:
        gr.force = vec(0., 0., 0.)

def verlet(grain_list, dt):
    '''Função que aplica a integração de verlet para atualizar as posições
    dos grãos.'''
    for gr in grain_list:
        a = gr.force/gr.mass
        gr.vel += (gr.acc + a) * (dt/2.)
        gr.pos += gr.vel * dt + 0.5*a*(dt**2.)
        gr.acc  = a

def apply_contacts(grain_list):
    '''Função que verifica se houve contato entre cada um dos pares de grãos
    por meio do cálculo de suas distâncias e, caso haja, aplica a função de
    força de contato contact().'''
    for gr1, gr2 in combinations(grain_list, 2):
        if norm(gr1.pos - gr2.pos) < gr1.radius + gr2.radius and gr1 != gr2:
            contact(gr1, gr2)

def time_loop(grain_list, dt, g, center, scale, rot, K):
    '''Função que gerencia o laço temporal, aplicando cada uma das funções
    previamente definidas em ordem para cada iteração.'''
    reset_force(grain_list)
    apply_gravity(grain_list, g)
    apply_contacts(grain_list)
    cube_walls(center , scale, grain_list, K, rot)
    verlet(grain_list, dt)

In [10]:
# Funções auxiliares de geração de grãos

def generate_grains_random(n, radius, density, sidelength):
    '''Função que gera uma lista de grãos com posições e velocidades aleatórias,
    respeitando a escala do cubo e o raio dos grãos.'''
    grain_list = []
    sidelength -= 1.5*radius
    for _ in range(n):
        gr = grain(vec(np.random.uniform(-sidelength/2, sidelength/2),
                       np.random.uniform(-sidelength/2, sidelength/2), 
                       np.random.uniform(0, sidelength)),
                    radius,
                    density,
                    vec(np.random.uniform(-1, 1),
                        np.random.uniform(-1, 1),
                        np.random.uniform(-1, 1)))
        grain_list.append(gr)
    return grain_list

def generate_grains_uniform(cube_scale: iter, n_max: iter, radius, density, randvel = False):
    '''Função que gera os grãos uniformemente distribuídos em uma grade, respeitando a escala
    do cubo e o raio dos grãos. É possível escolher uma escala menor que a do cubo e um número
    arbitrário de grãos, desde que caibam dentro do cubo com espaço. É possível também escolher
    se as velocidades iniciais serão todas nulas ou aleatórias.'''
    grain_list = []

    reduced_scale = cube_scale - 2*radius

    n_max = [min(int(r/radius), n) for r, n in zip(reduced_scale, n_max)]

    xs, ys, zs = reduced_scale

    for x in np.linspace(-xs, xs, n_max[0]):
        for y in np.linspace(-ys, ys, n_max[1]):
            for z in np.linspace(-zs, zs, n_max[2]):
                gr = grain(vec(x,y,z), radius, density, vec(0.,0.,0.))
                grain_list.append(gr)
    
    if randvel:
        for gr in grain_list:
            gr.vel = vec(np.random.uniform(-1, 1),
                        np.random.uniform(-1, 1),
                        np.random.uniform(-1, 1))

    return grain_list

In [31]:
# Funções auxiliares de movimentação do cubo

def shake(center: list, axis: int, T, dt, cycles_per_second= 1, scale=1):
    '''Função que move o cubo em um eixo, de forma a simular uma vibração.'''
    cube_pos = []

    center = np.array(center,dtype=float)

    total_angle = 2*np.pi * T * cycles_per_second

    n_elements = np.arange(0,T,dt).shape[0]

    angle_range = np.linspace(0, total_angle,n_elements)

    v = 10/(2*np.pi)

    for theta in angle_range:
        center[axis] = scale*np.sin(v*theta)
        cube_pos.append(center.copy())

    return cube_pos

In [32]:
cube_pos = shake(center = [0,0,0],
                 axis = 2,
                 T = 5,
                 dt =  0.001,
                 cycles_per_second=1,
                 scale=1)

In [34]:
cube_pos

[array([0., 0., 0.]),
 array([0.        , 0.        , 0.01000183]),
 array([0.        , 0.        , 0.02000267]),
 array([0.       , 0.       , 0.0300015]),
 array([0.        , 0.        , 0.03999733]),
 array([0.        , 0.        , 0.04998916]),
 array([0.        , 0.        , 0.05997599]),
 array([0.        , 0.        , 0.06995682]),
 array([0.        , 0.        , 0.07993065]),
 array([0.        , 0.        , 0.08989648]),
 array([0.        , 0.        , 0.09985332]),
 array([0.        , 0.        , 0.10980017]),
 array([0.        , 0.        , 0.11973604]),
 array([0.        , 0.        , 0.12965993]),
 array([0.        , 0.        , 0.13957085]),
 array([0.       , 0.       , 0.1494678]),
 array([0.       , 0.       , 0.1593498]),
 array([0.        , 0.        , 0.16921587]),
 array([0.      , 0.      , 0.179065]),
 array([0.        , 0.        , 0.18889622]),
 array([0.        , 0.        , 0.19870854]),
 array([0.        , 0.        , 0.20850099]),
 array([0.        , 0.     

## Execução de uma simulação de exemplo

In [35]:
# Parâmetros

T = 5

dt = 0.001

K = .001

g = vec(0.,0.,-9.81)

# Geometria do cubo

center = vec(0.,0.,0.)

scale = 1

rot = vec(0.,0.,0.)

time_steps = np.arange(0,T,dt)

# Aqui, no caso, o cubo não se move. Nota, ainda não foi implementada rotação do cubo.

# Grãos

cube_scale = vec(1.,1.,1.)

n_max = (3,3,3)

radius = 0.1

density = 1600

randvel = True

grain_list = generate_grains_uniform(cube_scale, n_max, radius, density, randvel=randvel)

In [36]:
# Execução da simulação

pos_list = [] # Lista de posições dos grãos

for i, _ in enumerate(time_steps):

    time_loop(grain_list, dt, g, center, scale, rot, K)
    
    temp_list = []
    
    for gr in grain_list:
        for pos in np.array(gr.pos):
            temp_list.append(pos)

    center = cube_pos[i]
    
    pos_list.append(temp_list.copy())

## Extração dos Dados

In [37]:
# Criando arquvios .csv completos para as posições dos grãos e do cubo

df = pd.DataFrame(pos_list)

df.to_csv('pos.csv')

cube_df = pd.DataFrame(cube_pos)

cube_df.to_csv('cube_pos.csv')

In [38]:
'''Precisamos cortar alguns frames para poder exportar ao Blender. O motivo para isso é que as animações
precisam ser renderizadas em no máximo 60 fps, mais do que isso é desperdício de poder computacional.
Então, cortamos a lista de quadros de forma a pegar um quadro a cada 60 quadros, ou seja, 60 quadros
por segundo. Essas listas serão carregadas no blender.'''

persec = 1/dt

fps = 60

frame_step = int(round(persec/fps,0))

cut_frames = pos_list[::frame_step]

df_frames = pd.DataFrame(cut_frames)

df_frames.to_csv('cut_frames.csv')

cube_cut_frames = cube_pos[::frame_step]

cube_df_frames = pd.DataFrame(cube_cut_frames)

cube_df_frames.to_csv('cube_cut_frames.csv')

Caso seja de interesse, posso disponibilizar o código interno no Blender que gera as animações.